In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
sound_path = '/content/gdrive/My Drive/learning/sound'

In [0]:
! git clone https://github.com/lithium0003/sound_separation.git
! cp sound_separation/ssWavenet/* .
! ln -s "{sound_path}" sound

In [0]:
! pip install pysoundfile joblib
! apt install libsndfile1

In [0]:
import json

def load_config(config_filepath):
    try:
        config_file = open(config_filepath, 'r')
    except IOError:
        print('No readable config file at path: ' + config_filepath)
        exit()
    else:
        with config_file:
            return json.load(config_file)

config = load_config('config.json')
config['training']['batch_size'] = 8*config['training']['batch_size']
config['training']['path'] = 'sound/sessions/wavenet/001'
config['tensorboard']['path'] = 'sound/logs/wavenet/001'
config['dataset'] = {
    'train' : {
        'vocal1': [
            'sound/data/train/part1_12.wav'
        ],
        'vocal2': [
            'sound/data/train/part2_12.wav'
        ],
         'mixture': [
            'sound/data/train/mixture_12.wav'
        ],
    },
    'val' : {
        'vocal1': [
            'sound/data/val/part1_12.wav'
        ],
        'vocal2': [
            'sound/data/val/part2_12.wav'
        ],
         'mixture': [
            'sound/data/val/mixture_12.wav'
        ],
    },
}

In [0]:
import tensorflow as tf

import models
import datasets

def training(config):
    #exp = Experiment('Wavenet Separation')

    model = models.SeparationWavenet(config, True)
    dataset = datasets.VoiceDataset(config, model).load_dataset()

    num_steps_train = config['training']['num_steps_train']
    num_steps_val = config['training']['num_steps_test']
    train_set_generator = dataset.get_random_batch_generator('train')
    val_set_generator = dataset.get_random_batch_generator('val')

    model.fit_model(train_set_generator, num_steps_train, val_set_generator, num_steps_val,
                    config['training']['num_epochs'])

    #exp.end()

 
training(config)